In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
def distribute_water(glasses):
    # Ali distributes the volume randomly among the glasses
    for i in range(20):  
        glasses[random.randint(0, len(glasses) - 1)] += 0.5/20 
    return glasses

def remove_water(glasses):
    # Beth randomly selects two adjacent glasses to empty
    start_glass = random.randint(0, len(glasses) - 1)
    glasses[start_glass] = 0
    glasses[(start_glass + 1) % len(glasses)] = 0 
    return glasses

def simulate_game(turns,num_glasses):
    glasses = [0]*num_glasses
    for t in range(turns):
        glasses = distribute_water(glasses)
        glasses = remove_water(glasses)
        if any(glass > 1 for glass in glasses):  
            return t+1
    return 0


In [3]:
def calculate_probabilities(turns, simulations, num_glasses):
    overflow_counts = [0] * (turns + 1) 

    for i in range(simulations):
        turn = simulate_game(turns, num_glasses)
        overflow_counts[turn] += 1

    probabilities = [count / simulations for count in overflow_counts]
    return probabilities

In [ ]:
glasses = [0, 0, 0, 0, 0]
num_glasses = 5
turns = 10
simulations = 100
probabilities = calculate_probabilities(turns, simulations, num_glasses)

for t in range(1, turns + 1):
    print(f"Probability of overflow on turn {t}: {probabilities[t]:.4f}")
print(f"Probability of no overflow within {turns} turns: {probabilities[0]:.4f}")

In [4]:
turns = 50
simulations = 1000

fig, ax = plt.subplots()
ax.set_xlim(0, turns)
ax.set_ylim(0, 1.1)  
ax.set_xlabel('Number of Turns')
ax.set_ylabel('Cumulative Probability of Overflow')
ax.grid()
line, = ax.plot([], [], lw=2)
title = ax.set_title('')

def animate(num_glasses):
    probabilities = calculate_probabilities(turns, simulations, num_glasses)
    # Convert the probabilities list to a cumulative one
    cumulative_probabilities = np.cumsum(probabilities[1:]) 
    x = np.arange(1, turns + 1)
    y = cumulative_probabilities
    line.set_data(x, y)
    title.set_text(f'Cumulative Probability of Overflow for {num_glasses} Glasses')
    return line, title


ani = FuncAnimation(fig, animate, frames=range(2, 11), interval=800)  


plt.show()

In [5]:
turns = 50
simulations = 1000
max_glasses = 10  

for num_glasses in range(2, max_glasses + 1):  
    # Calculate probabilities
    probabilities = calculate_probabilities(turns, simulations, num_glasses)
    cumulative_probabilities = np.cumsum(probabilities[1:])  # Skip the first value (no overflow)
    
    plt.figure(figsize=(8, 6))
    plt.plot(np.arange(1, turns + 1), cumulative_probabilities, lw=2, label=f'{num_glasses} Glasses')
    plt.xlim(0, turns)
    plt.ylim(0, 1.1)
    plt.xlabel('Number of Turns')
    plt.ylabel('Cumulative Probability of Overflow')
    plt.title(f'Cumulative Probability of Overflow for {num_glasses} Glasses')
    plt.legend()
    plt.grid()
    plt.show()

In [6]:
def remove_water_stategy1(glasses):
    max_index = glasses.index(max(glasses))  
    # Determine the indices of the adjacent glasses, considering the circular arrangement
    left_index = (max_index - 1) % len(glasses)
    right_index = (max_index + 1) % len(glasses)
    
    if glasses[left_index] > glasses[right_index]:
        glasses[left_index] = 0 
    else:
        glasses[right_index] = 0  
    
    glasses[max_index] = 0  
    return glasses

In [7]:
def remove_water_strategy2(glasses):
    max_sum = 0 
    index = 0 

    for i in range(len(glasses)):
        j = (i + 1) % len(glasses)
        current_sum = glasses[i] + glasses[j]
        if current_sum > max_sum:
            max_sum = current_sum
            index = i

    # Empty the two glasses with the maximum total volume
    glasses[index] = 0
    glasses[(index + 1) % len(glasses)] = 0

    return index, max_sum

In [8]:
# Ali's water distribute strategy for five glasses
def distribute_water_ali_strategy5(glasses, increment):
    total_water = glasses[0] + glasses[2] + glasses[3]
    water_for_glass_1 = (total_water+1/2)/2 - 2*increment
    water_for_glass_3_4 = (total_water+1/2)/4+increment

    glasses[0] = water_for_glass_1
    glasses[2] = water_for_glass_3_4
    glasses[3] = water_for_glass_3_4
    
    return glasses

In [9]:
def simulate_game(turns, num_glasses, increment):
    glasses = [0] * num_glasses
    for t in range(turns):
        glasses = distribute_water_ali_strategy5(glasses, increment)
        glasses = remove_water_stategy1(glasses)
        if all(glass > 0.5 for glass in glasses):
            return t + 1
    return 0  


max_turns = 50
simulations = 100
num_glasses = 5
increments = np.linspace(0.0001, 0.1, 200)  

fig, ax = plt.subplots()
line, = ax.plot([], [], 'r-', lw=2)
ax.set_xlim(0, max_turns)
ax.set_ylim(0, 1)
ax.set_xlabel('Number of Turns')
ax.set_ylabel('Probability of Ali Causing an Overflow')
ax.grid()
title = ax.set_title('')

def init():
    line.set_data([], [])
    title.set_text('')
    return line, title

def animate(i):
    increment = increments[i]
    probabilities = [sum(simulate_game(turn, num_glasses, increment) for _ in range(simulations)) / simulations for turn in range(1, max_turns + 1)]
    x = np.arange(1, max_turns + 1)
    y = probabilities
    line.set_data(x, y)
    title.set_text(f'Increment: {increment:.2f}')
    return line, title

ani = FuncAnimation(fig, animate, init_func=init, frames=len(increments), blit=True, interval=200)

plt.title('Probability of Ali Causing an Overflow Over Turns (Varying Increment)')
plt.show()

In [11]:
def distribute_water_evenly_strategy_check3(glasses):
    total_water = sum(glasses) + 0.5 
    new_water_level = total_water / len(glasses)
    
    for i in range(len(glasses)):
        glasses[i] = new_water_level
    return glasses

def simulate_game3(turns, num_glasses):
    glasses = [0] * num_glasses
    for t in range(turns):
        glasses = distribute_water_evenly_strategy_check3(glasses)
        print(f"After Ali's move: {glasses}")
        glasses = remove_water_stategy1(glasses)
        print(f"After Beth's move: {glasses}")
        if all(glass > 0.5 for glass in glasses):
            return t + 1 
    return 0 

turns = 10
num_glasses = 3
result = simulate_game3(turns, num_glasses)
if result == 0:
    print ('Ali will not have a glass with more than half a pint by Round10')
else:
    print(f"Ali have a glass with more than half a pint by Round: {result}")

After Ali's move: [0.16666666666666666, 0.16666666666666666, 0.16666666666666666]
After Beth's move: [0, 0, 0.16666666666666666]
After Ali's move: [0.2222222222222222, 0.2222222222222222, 0.2222222222222222]
After Beth's move: [0, 0, 0.2222222222222222]
After Ali's move: [0.24074074074074073, 0.24074074074074073, 0.24074074074074073]
After Beth's move: [0, 0, 0.24074074074074073]
After Ali's move: [0.24691358024691357, 0.24691358024691357, 0.24691358024691357]
After Beth's move: [0, 0, 0.24691358024691357]
After Ali's move: [0.24897119341563786, 0.24897119341563786, 0.24897119341563786]
After Beth's move: [0, 0, 0.24897119341563786]
After Ali's move: [0.2496570644718793, 0.2496570644718793, 0.2496570644718793]
After Beth's move: [0, 0, 0.2496570644718793]
After Ali's move: [0.2498856881572931, 0.2498856881572931, 0.2498856881572931]
After Beth's move: [0, 0, 0.2498856881572931]
After Ali's move: [0.24996189605243102, 0.24996189605243102, 0.24996189605243102]
After Beth's move: [0, 0, 0

In [12]:

def distribute_water_evenly_strategy_check4(glasses):
    total_water = sum(glasses) + 0.5  
    new_water_level = total_water / 2  
    for i in [0, 2]: 
        glasses[i] = new_water_level
    return glasses

def simulate_game4(turns, num_glasses):
    glasses = [0] * num_glasses
    for t in range(turns):
        glasses = distribute_water_evenly_strategy_check4(glasses)
        print(f"After Ali's move: {glasses}")
        glasses = remove_water_stategy1(glasses)
        print(f"After Beth's move: {glasses}")
        if any(glass > 0.5 for glass in glasses):
            return t + 1
    return 0


turns = 10
num_glasses = 4
result = simulate_game4(turns, num_glasses)
if result == 0:
    print ('Ali will not have a glass with more than half a pint by Round10')
else:
    print(f"Ali have a glass with more than half a pint by Round: {result}")

After Ali's move: [0.25, 0, 0.25, 0]
After Beth's move: [0, 0, 0.25, 0]
After Ali's move: [0.375, 0, 0.375, 0]
After Beth's move: [0, 0, 0.375, 0]
After Ali's move: [0.4375, 0, 0.4375, 0]
After Beth's move: [0, 0, 0.4375, 0]
After Ali's move: [0.46875, 0, 0.46875, 0]
After Beth's move: [0, 0, 0.46875, 0]
After Ali's move: [0.484375, 0, 0.484375, 0]
After Beth's move: [0, 0, 0.484375, 0]
After Ali's move: [0.4921875, 0, 0.4921875, 0]
After Beth's move: [0, 0, 0.4921875, 0]
After Ali's move: [0.49609375, 0, 0.49609375, 0]
After Beth's move: [0, 0, 0.49609375, 0]
After Ali's move: [0.498046875, 0, 0.498046875, 0]
After Beth's move: [0, 0, 0.498046875, 0]
After Ali's move: [0.4990234375, 0, 0.4990234375, 0]
After Beth's move: [0, 0, 0.4990234375, 0]
After Ali's move: [0.49951171875, 0, 0.49951171875, 0]
After Beth's move: [0, 0, 0.49951171875, 0]
Ali will not have a glass with more than half a pint by Round10


In [13]:
def distribute_water_evenly_strategy_check6(glasses):
    total_water = sum(glasses) + 0.5 
    new_water_level = total_water / 3  
    for i in [0, 2, 4]:
        glasses[i] = new_water_level
    return glasses

def simulate_game6(turns, num_glasses):
    glasses = [0] * num_glasses
    for t in range(turns):
        glasses = distribute_water_evenly_strategy_check6(glasses)
        print(f"After Ali's move: {glasses}")
        glasses = remove_water_stategy1(glasses)
        print(f"After Beth's move: {glasses}")
        count_over_quarter = sum(glass > 0.25 for glass in glasses)
        
        # Check if at least two glasses have more than 1/4 units of water
        if count_over_quarter >= 2:
            return t + 1  
    return 0 


turns = 20
num_glasses = 6
result = simulate_game6(turns, num_glasses)
print(f"Ali have more than two 1/4 by Round: {result}")

After Ali's move: [0.16666666666666666, 0, 0.16666666666666666, 0, 0.16666666666666666, 0]
After Beth's move: [0, 0, 0.16666666666666666, 0, 0.16666666666666666, 0]
After Ali's move: [0.27777777777777773, 0, 0.27777777777777773, 0, 0.27777777777777773, 0]
After Beth's move: [0, 0, 0.27777777777777773, 0, 0.27777777777777773, 0]
Ali have more than two 1/4 by Round: 2
